# 数据库示例

## 1、数据库连接

In [2]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
sqluser = 'postgres'
dbname = 'mimic'
schema_name = 'mimiciii'
con = psycopg2.connect(dbname=dbname, user=sqluser,password='0')
query_schema = 'set search_path to ' + schema_name + ';'

## 2、仅进入一次ICU的病例编号与相关信息

In [6]:
query = query_schema + """
with co as
(
SELECT icu.subject_id, icu.hadm_id,icu.icustay_id,icu.first_careunit,icu.intime,icu.outtime,icu.los
FROM mimiciii.icustays icu
)
select * from co where co.subject_id in (
    select co.subject_id from co group by co.subject_id having count(*)=1
) order by co.subject_id;
"""
df = pd.read_sql_query(query, con)
df.head()

,subject_id,hadm_id,icustay_id,first_careunit,intime,outtime,los
0,2,163353,243653,NICU,2138-07-17 21:20:07,2138-07-17 23:32:21,0.0918
1,3,145834,211552,MICU,2101-10-20 19:10:11,2101-10-26 20:43:09,6.0646
2,4,185777,294638,MICU,2191-03-16 00:29:31,2191-03-17 16:46:31,1.6785
3,5,178980,214757,NICU,2103-02-02 06:04:24,2103-02-02 08:06:00,0.0844
4,6,107064,228232,SICU,2175-05-30 21:30:54,2175-06-03 13:39:54,3.6729


## 3、提取年龄过程（挑出成年人）

In [4]:
query = query_schema + """
with co as(
SELECT * FROM(
select a.subject_id,a.dob,b.intime,extract(year from b.intime)-extract(year from a.dob) as age 
from  mimiciii.patients as a ,mimiciii.icustays as b
where a.subject_id=b.subject_id)C
where c.subject_id in (select subject_id from mimiciii.onehospital)
) 
select subject_id,age from co 
where age>15
order by co.subject_id
"""
df = pd.read_sql_query(query, con)
df.head()

,subject_id,age
0,3,76.0
1,4,48.0
2,6,66.0
3,9,41.0
4,11,50.0


## 4、添加结果标签（幸存与死亡）

In [7]:
query = query_schema + """
/*CREATE MATERIALIZED VIEW mimiciii.inhospitaldeath as 
with co as(
select * from(
select a.subject_id,a.expire_flag,b.hospital_expire_flag
from mimiciii.patients as a,mimiciii.onehospital as b
where a.subject_id=b.subject_id)c
where c.subject_id in (select subject_id from mimiciii.age) 
order by c.subject_id
)

select * from co
where co.expire_flag=1 and co.hospital_expire_flag=1*/
SELECT * FROM(
select a.subject_id,a.dod,b.intime,(CASE WHEN a.dod < b.intime + interval '30' day THEN 1 ELSE 0 END) as typ 
from  mimiciii.patients as a ,mimiciii.icustays as b
where a.subject_id=b.subject_id)C
where c.subject_id in (select subject_id from mimiciii.inhospitaldeath) and c.typ=1
"""
df = pd.read_sql_query(query, con)
df.head()

,subject_id,dod,intime,typ
0,250,2188-11-22,2188-11-12 09:25:47,1
1,268,2198-02-18,2198-02-14 23:27:38,1
2,669,2182-07-31,2182-07-31 04:28:16,1
3,705,2156-08-26,2156-08-23 00:44:29,1
4,710,2182-02-28,2182-02-18 17:36:06,1


## 5、提取FiO2变量

In [9]:
query = query_schema + """
-- This query pivots the vital signs for the first 24 hours of a patient's stay
-- Vital signs include heart rate, blood pressure, respiration rate, and temperature

--DROP MATERIALIZED VIEW IF EXISTS vitalsfirstday1 CASCADE;
--create materialized view mimiciii.vitalsfio22 as
SELECT pvt.subject_id, pvt.hadm_id, pvt.icustay_id

-- Easier names

, min(case when VitalID = 11 then valuenum else null end) as FiO2_Min
, max(case when VitalID = 11 then valuenum else null end) as FiO2_Max
, avg(case when VitalID = 11 then valuenum else null end) as FiO2_Mean

FROM  (
  select ie.subject_id, ie.hadm_id, ie.icustay_id
  , case

    when itemid in (190,3420,3422,223835) and valuenum > 0 and valuenum <= 100 then 11 -- FiO2



    else null end as VitalID

  , case when itemid in (223835) then 
                    case  
                        WHEN ((valuenum > 0) AND (valuenum <= 1)) THEN (valuenum * 100)
                        WHEN ((valuenum > 1) AND (valuenum < 21)) THEN NULL
                        WHEN ((valuenum >= 21) AND (valuenum <= 100)) THEN valuenum
                        ELSE NULL
                    END
                    WHEN (itemid = ANY (ARRAY[3420, 3422])) THEN valuenum
                    WHEN ((itemid = 190) AND (valuenum > 0.20) AND (valuenum < 1)) THEN (valuenum * 100)
                    ELSE NULL
    END AS valuenum


  from mimiciii.icustays ie
  left join mimiciii.chartevents ce
  on ie.subject_id = ce.subject_id and ie.hadm_id = ce.hadm_id and ie.icustay_id = ce.icustay_id
  and ce.charttime between ie.intime + interval '12' hour and ie.intime + interval '1' day
  -- exclude rows marked as error
  and ce.error IS DISTINCT FROM 1
  where ce.itemid in
  (

  
  190, -- "FiO2 Set"
  3420, -- "FiO2"
  3422, -- "FiO2 [Meas]"
  223835 -- "Inspired O2 Fraction"

  )
) pvt
group by pvt.subject_id, pvt.hadm_id, pvt.icustay_id
order by pvt.subject_id, pvt.hadm_id, pvt.icustay_id;
"""
df = pd.read_sql_query(query, con)
df.head()

,subject_id,hadm_id,icustay_id,fio2_min,fio2_max,fio2_mean
0,3,145834,211552,40.000001,50.000000,42.500000
1,8,159514,262299,NaN,NaN,NaN
2,9,150750,220597,60.000002,60.000002,60.000002
3,12,112213,232669,40.000001,50.000000,44.000000
4,13,143045,263738,50.000000,50.000000,50.000000


## 6、提取其他特征值

In [13]:
query = query_schema + """
-- This query pivots the vital signs for the first 24 hours of a patient's stay
-- Vital signs include heart rate, blood pressure, respiration rate, and temperature

--DROP MATERIALIZED VIEW IF EXISTS vitalsfirstday1 CASCADE;
--create materialized view mimiciii.vitalsfirstday2 as
SELECT pvt.subject_id, pvt.hadm_id, pvt.icustay_id

-- Easier names
, min(case when VitalID = 1 then valuenum else null end) as HeartRate_Min
, max(case when VitalID = 1 then valuenum else null end) as HeartRate_Max
, avg(case when VitalID = 1 then valuenum else null end) as HeartRate_Mean
, min(case when VitalID = 2 then valuenum else null end) as SysBP_Min
, max(case when VitalID = 2 then valuenum else null end) as SysBP_Max
, avg(case when VitalID = 2 then valuenum else null end) as SysBP_Mean
, min(case when VitalID = 3 then valuenum else null end) as DiasBP_Min
, max(case when VitalID = 3 then valuenum else null end) as DiasBP_Max
, avg(case when VitalID = 3 then valuenum else null end) as DiasBP_Mean
, min(case when VitalID = 4 then valuenum else null end) as MeanBP_Min
, max(case when VitalID = 4 then valuenum else null end) as MeanBP_Max
, avg(case when VitalID = 4 then valuenum else null end) as MeanBP_Mean
, min(case when VitalID = 5 then valuenum else null end) as RespRate_Min
, max(case when VitalID = 5 then valuenum else null end) as RespRate_Max
, avg(case when VitalID = 5 then valuenum else null end) as RespRate_Mean
, min(case when VitalID = 6 then valuenum else null end) as TempC_Min
, max(case when VitalID = 6 then valuenum else null end) as TempC_Max
, avg(case when VitalID = 6 then valuenum else null end) as TempC_Mean
, min(case when VitalID = 7 then valuenum else null end) as SpO2_Min
, max(case when VitalID = 7 then valuenum else null end) as SpO2_Max
, avg(case when VitalID = 7 then valuenum else null end) as SpO2_Mean
, min(case when VitalID = 8 then valuenum else null end) as NISysBP_Min
, max(case when VitalID = 8 then valuenum else null end) as NISysBP_Max
, avg(case when VitalID = 8 then valuenum else null end) as NISysBP_Mean
, min(case when VitalID = 9 then valuenum else null end) as NIDiasBP_Min
, max(case when VitalID = 9 then valuenum else null end) as NIDiasBP_Max
, avg(case when VitalID = 9 then valuenum else null end) as NIDiasBP_Mean
, min(case when VitalID = 10 then valuenum else null end) as NIMeanBP_Min
, max(case when VitalID = 10 then valuenum else null end) as NIMeanBP_Max
, avg(case when VitalID = 10 then valuenum else null end) as NIMeanBP_Mean

FROM  (
  select ie.subject_id, ie.hadm_id, ie.icustay_id
  , case
    when itemid in (211,220045) and valuenum > 0 and valuenum < 300 then 1 -- HeartRate
    when itemid in (51,6701,220050) and valuenum > 0 and valuenum < 400 then 2 -- SysBP
    when itemid in (8368,8555,220051) and valuenum > 0 and valuenum < 300 then 3 -- DiasBP
    when itemid in (52,6702,220052,225312) and valuenum > 0 and valuenum < 300 then 4 -- MeanBP
    when itemid in (615,618,220210,224690) and valuenum > 0 and valuenum < 70 then 5 -- RespRate
    when itemid in (223761,678) and valuenum > 70 and valuenum < 120  then 6 -- TempF, converted to degC in valuenum call
    when itemid in (223762,676) and valuenum > 10 and valuenum < 50  then 6 -- TempC
    when itemid in (646,220277) and valuenum > 0 and valuenum <= 100 then 7 -- SpO2
    when itemid in (442,455,220179) and valuenum > 0 and valuenum < 400 then 8 -- NISysBP
    when itemid in (8440,8441,220180) and valuenum > 0 and valuenum < 300 then 9 -- NIDiasBP
    when itemid in (456,443,220181) and valuenum > 0 and valuenum < 300 then 10 -- NIMeanBP

    else null end as VitalID
      -- convert F to C
  , case when itemid in (223761,678) then (valuenum-32)/1.8 else valuenum end as valuenum

  from mimiciii.icustays ie
  left join mimiciii.chartevents ce
  on ie.subject_id = ce.subject_id and ie.hadm_id = ce.hadm_id and ie.icustay_id = ce.icustay_id
  and ce.charttime between ie.intime + interval '12' hour and ie.intime + interval '1' day
  -- exclude rows marked as error
  and ce.error IS DISTINCT FROM 1
  where ce.itemid in
  (
  -- HEART RATE
  211, --"Heart Rate"
  220045, --"Heart Rate"

  -- Systolic/diastolic

  51, --	Arterial BP [Systolic]
  442, --	Manual BP [Systolic]
  455, --	NBP [Systolic]
  6701, --	Arterial BP #2 [Systolic]
  220179, --	Non Invasive Blood Pressure systolic
  220050, --	Arterial Blood Pressure systolic

  8368, --	Arterial BP [Diastolic]
  8440, --	Manual BP [Diastolic]
  8441, --	NBP [Diastolic]
  8555, --	Arterial BP #2 [Diastolic]
  220180, --	Non Invasive Blood Pressure diastolic
  220051, --	Arterial Blood Pressure diastolic


  -- MEAN ARTERIAL PRESSURE
  456, --"NBP Mean"
  52, --"Arterial BP Mean"
  6702, --	Arterial BP Mean #2
  443, --	Manual BP Mean(calc)
  220052, --"Arterial Blood Pressure mean"
  220181, --"Non Invasive Blood Pressure mean"
  225312, --"ART BP mean"

  -- RESPIRATORY RATE
  618,--	Respiratory Rate
  615,--	Resp Rate (Total)
  220210,--	Respiratory Rate
  224690, --	Respiratory Rate (Total)


  -- SpO2, peripheral
  646, 220277,

  -- TEMPERATURE
  223762, -- "Temperature Celsius"
  676,	-- "Temperature C"
  223761, -- "Temperature Fahrenheit"
  678 --	"Temperature F"
  )
) pvt
group by pvt.subject_id, pvt.hadm_id, pvt.icustay_id
order by pvt.subject_id, pvt.hadm_id, pvt.icustay_id;

"""
df = pd.read_sql_query(query, con)
df.head()

,subject_id,hadm_id,icustay_id,heartrate_min,heartrate_max,heartrate_mean,sysbp_min,sysbp_max,sysbp_mean,diasbp_min,...,spo2_mean,nisysbp_min,nisysbp_max,nisysbp_mean,nidiasbp_min,nidiasbp_max,nidiasbp_mean,nimeanbp_min,nimeanbp_max,nimeanbp_mean
0,3,145834,211552,75.0,96.0,85.500000,92.0,119.0,107.00,52.0,...,98.272727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,185777,294638,85.0,99.0,91.363636,NaN,NaN,NaN,NaN,...,97.250000,113.0,139.0,125.900000,69.0,81.0,75.800000,85.666702,100.333000,92.499970
2,6,107064,228232,77.0,100.0,86.384615,127.0,187.0,168.00,53.0,...,98.538462,130.0,159.0,144.111111,44.0,72.0,52.555556,72.666702,96.333298,83.074078
3,8,159514,262299,125.0,148.0,135.384615,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9,150750,220597,90.0,111.0,100.583333,136.0,185.0,157.25,66.0,...,98.083333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 7、建立基本信息表格

In [15]:
query = query_schema + """
/*CREATE MATERIALIZED VIEW mimiciii.inhospital as 
with co as(
select * from(
select a.subject_id,a.expire_flag,b.hospital_expire_flag,a.dod
from mimiciii.patients as a,mimiciii.onehospital as b
where a.subject_id=b.subject_id)c
where c.subject_id in (select subject_id from mimiciii.age)
order by c.subject_id
)

select co.subject_id,co.expire_flag,co.hospital_expire_flag,co.dod,c.intime
from co,mimiciii.icustays as c
where co.subject_id=c.subject_id*/

/*CREATE MATERIALIZED VIEW mimiciii.deathlabel as 
select bg.subject_id,bg.expire_flag,bg.hospital_expire_flag,bg.dod,bg.intime,
  (
case when bg.expire_flag=0 then 0
     when bg.expire_flag=1 then
      case when bg.hospital_expire_flag=0 then
       case when bg.dod < bg.intime + interval '30' day THEN 2
            when bg.dod >= bg.intime + interval '30' day THEN 0
         else null end

        when bg.hospital_expire_flag=1 then
        case  when bg.dod < bg.intime + interval '30' day THEN 1
               when bg.dod >= bg.intime + interval '30' day THEN 0
         else null end
         else null end
    else null end
             ) as death
 from mimiciii.inhospital as bg */
with co as(
select * from(
select a.subject_id,a.hadm_id,a.icustay_id,(case when b.age=300 then 91 else age end) as age,
       (case when b.gender='M' then 1 else 0 end) as gender,
       (case when b.first_careunit='CCU' then 1 
             when b.first_careunit='CSRU' then 2
             when b.first_careunit='MICU' then 3
             when b.first_careunit='SICU' then 4
             when b.first_careunit='TSICU' then 5
             else 0 end) as first_careunit
from mimiciii.oneicu as a,mimiciii.baseinfo1 as b
where a.subject_id=b.subject_id)c
where c.subject_id in (select subject_id from mimiciii.age)
)

select co.subject_id,co.hadm_id,co.icustay_id,co.age,co.gender,co.first_careunit,cc.death
from co,mimiciii.deathlabel as cc
where co.subject_id=cc.subject_id and (death=0 or death=1)
"""
df = pd.read_sql_query(query, con)
df.head()

,subject_id,hadm_id,icustay_id,age,gender,first_careunit,death
0,3,145834,211552,76.0,1,3,0
1,4,185777,294638,48.0,0,3,0
2,6,107064,228232,66.0,0,4,0
3,9,150750,220597,41.0,1,3,1
4,11,194540,229441,50.0,0,4,0


## 8、建立最终特征值表

In [17]:
query = query_schema + """
/*CREATE MATERIALIZED VIEW mimiciii.finalbase as
with co as(
SELECT a.icustay_id,a.weight, b.height,(a.weight/(b.height/100)^2) as BMI
FROM mimiciii.weightfirstday as a,mimiciii.heightfirstday as b
where a.icustay_id=b.icustay_id
)

,we as(
select m.subject_id,m.hadm_id,m.icustay_id,m.age,m.gender,m.first_careunit,m.death,uofirstday.urineoutput from mimiciii.baseinformation as m
left join mimiciii.uofirstday on
m.subject_id=uofirstday.subject_id and m.hadm_id=uofirstday.hadm_id and m.icustay_id=uofirstday.icustay_id
)

select we.subject_id,we.hadm_id,we.icustay_id,we.age,we.gender,we.first_careunit,we.death,we.urineoutput,co.BMI from we
left join co on
we.icustay_id=co.icustay_id
order by subject_id*/

/*CREATE MATERIALIZED VIEW mimiciii.vari as
SELECT a.subject_id,a.heartrate_min as qHRmin,a.heartrate_max as qHRmax,a.heartrate_mean as qHRmean,b.heartrate_min as hHRmin,b.heartrate_max as hHRmax,b.heartrate_mean as hHRmean,
                    a.resprate_min as qrespmin,a.resprate_max as qrespmax,a.resprate_mean as qrespmean, b.resprate_min as hrespmin,b.resprate_max as hrespmax,b.resprate_mean as hrespmean,
                    a.tempc_min as qtempmin,a.tempc_max as qtempmax,a.tempc_mean as qtempmean,b.tempc_min as htempmin,b.tempc_max as htempmax,b.tempc_mean as htempmean,
                    a.spo2_min as qspo2min,a.spo2_max as qspo2max,a.spo2_mean as qspo2mean,b.spo2_min as hspo2min,b.spo2_max as hspo2max,b.spo2_mean as hspo2mean,
                    a.nisysbp_min as qnisysbp_min,a.nisysbp_max as qnisysbp_max,a.nisysbp_mean as qnisysbp_mean,b.nisysbp_min as hnisysbp_min,b.nisysbp_max as hnisysbp_max,b.nisysbp_mean as hnisysbp_mean,
                    a.nidiasbp_min as qnidiasbp_min,a.nidiasbp_max as qnidiasbp_max,a.nidiasbp_mean as qnidiasbp_mean,b.nidiasbp_min as hnidiasbp_min,b.nidiasbp_max as hnidiasbp_max,b.nidiasbp_mean as hnidiasbp_mean,
                    a.nimeanbp_min as qnimeanbp_min,a.nimeanbp_max as qnimeanbp_max,a.nimeanbp_mean as qnimeanbp_mean,b.nimeanbp_min as hnimeanbp_min,b.nimeanbp_max as hnimeanbp_max,b.nimeanbp_mean as hnimeanbp_mean
from mimiciii.vitalsfirstday1 as a,mimiciii.vitalsfirstday2 as b
where a.subject_id=b.subject_id and a.hadm_id=b.hadm_id and a.icustay_id=b.icustay_id*/

/*CREATE MATERIALIZED VIEW mimiciii.vari1 as
with co as(
select a.subject_id,a.age,a.gender,a.first_careunit as icutype,a.death,a.urineoutput,a.bmi,b.mingcs as qgcs,c.fio2_min as qfio2min,c.fio2_max as qfio2max,c.fio2_mean as qfio2mean
from mimiciii.finalbase as a
left join mimiciii.gcsfirstday1 as b  on a.subject_id=b.subject_id and a.hadm_id=b.hadm_id and a.icustay_id=b.icustay_id
left join mimiciii.vitalsfio21 as c on a.subject_id=c.subject_id and a.hadm_id=c.hadm_id and a.icustay_id=c.icustay_id
)



select a.subject_id,a.age,a.gender,a.icutype,a.death,a.urineoutput,a.bmi,a.qgcs,b.mingcs as hgcs,a.qfio2min,a.qfio2max,a.qfio2mean,c.fio2_min as hfio2min,c.fio2_max as hfio2max,c.fio2_mean as hfio2mean
from co as a
left join mimiciii.gcsfirstday2 as b  on a.subject_id=b.subject_id 
left join mimiciii.vitalsfio22 as c on a.subject_id=c.subject_id 
order by a.subject_id*/

--CREATE table mimiciii.final as
select a.subject_id,a.age,a.gender,a.icutype,a.death,a.urineoutput,a.bmi,a.qgcs,a.hgcs,a.qfio2min,a.qfio2max,a.qfio2mean,a.hfio2min,a.hfio2max,a.hfio2mean,
       b.qHRmin,b.qHRmax,b.qHRmean,b.hHRmin,b.hHRmax,b.hHRmean,
       b.qrespmin,b.qrespmax,b.qrespmean,b.hrespmin,b.hrespmax,b.hrespmean,
       b.qtempmin,b.qtempmax,b.qtempmean,b.htempmin,b.htempmax,b.htempmean,
       b.qspo2min,b.qspo2max,b.qspo2mean,b.hspo2min,b.hspo2max,b.hspo2mean,
       b.qnisysbp_min,b.qnisysbp_max,b.qnisysbp_mean,b.hnisysbp_min,b.hnisysbp_max,b.hnisysbp_mean, 
       b.qnidiasbp_min,b.qnidiasbp_max,b.qnidiasbp_mean,b.hnidiasbp_min,b.hnidiasbp_max,b.hnidiasbp_mean,
       b.qnimeanbp_min,b.qnimeanbp_max,b.qnimeanbp_mean,b.hnimeanbp_min,b.hnimeanbp_max,b.hnimeanbp_mean
from mimiciii.vari1 as a
left join mimiciii.vari as b on a.subject_id=b.subject_id 
order by a.subject_id
"""
df = pd.read_sql_query(query, con)
df.head()

,subject_id,age,gender,icutype,death,urineoutput,bmi,qgcs,hgcs,qfio2min,...,qnidiasbp_mean,hnidiasbp_min,hnidiasbp_max,hnidiasbp_mean,qnimeanbp_min,qnimeanbp_max,qnimeanbp_mean,hnimeanbp_min,hnimeanbp_max,hnimeanbp_mean
0,3,76.0,1,3,0,497.0,30.187677,15.0,15.0,50.000000,...,41.909091,NaN,NaN,NaN,40.000000,97.666702,61.303036,NaN,NaN,NaN
1,4,48.0,0,3,0,2150.0,NaN,15.0,15.0,50.000000,...,61.444444,69.0,81.0,75.800000,69.000000,88.000000,77.370378,85.666702,100.333000,92.499970
2,6,66.0,0,4,0,1940.0,NaN,15.0,15.0,NaN,...,41.500000,44.0,72.0,52.555556,72.666702,73.000000,72.833351,72.666702,96.333298,83.074078
3,9,41.0,1,3,1,887.0,31.095741,15.0,15.0,60.000002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,50.0,0,4,0,1050.0,NaN,15.0,13.0,NaN,...,55.750000,38.0,58.0,49.333333,64.000000,89.000000,75.416667,58.000000,76.000000,67.416667
